In [1]:
import warnings
import numpy as np
from numpy import cos, sin, pi
from ipyvolume import pylab as p3
import math
from ipywidgets import widgets
from ipywidgets import FloatRangeSlider, Dropdown, FloatSlider, FloatLogSlider, ColorPicker, Box, VBox, Layout, Label
from ipywidgets import interact, interactive, fixed, interact_manual

try:
    import scipy.ndimage
    import scipy.special
except:
    pass
    
def klein_bottle(
    endpoint=True,
    tab=None,
    index=0
):
    # http://paulbourke.net/geometry/klein/
    u = np.linspace(0, 2 * pi, num=50, endpoint=endpoint)
    v = np.linspace(0, 2 * pi, num=50, endpoint=endpoint)
    u, v = np.meshgrid(u, v)
    r = 4 * (1 - cos(u) / 2)
    x = 6 * cos(u) * (1 + sin(u)) + r * cos(u) * cos(v) * (u < pi) + r * cos(v + pi) * (u >= pi)
    y = 16 * sin(u) + r * sin(u) * cos(v) * (u < pi)
    z = r * sin(v)
    mesh = p3.plot_mesh(
                x,
                y,
                z,
                wireframe=False,
                wrapx=not endpoint,
                wrapy=not endpoint,
                u=u / (2 * np.pi),
                v=v / (2 * np.pi))
    p3.setup_material_widgets(mesh=mesh, tab=tab, index=index)
    return mesh
    
def worldplane(
    tab=None,
    index=0
):
    k = 20
    h = -15
    tx = np.array([k, -k, -k, k])
    tz = np.array([k, k, -k, -k])
    ty = np.array([h, h, h, h])
    
    tri = [(0, 1, 2), (0, 2, 3)]
    p = p3.plot_trisurf(
        tx, 
        ty,
        tz, 
        triangles=tri)
    p3.setup_material_widgets(mesh=p, tab=tab, index=index)
    return p
    
def test_surface(
    tab=None,
    index=0):
    X = np.arange(-10, 10, 0.25*1)-10
    Y = np.arange(-10, 10, 0.25*1)
    X, Y = np.meshgrid(X, Y)
    R = np.sqrt(X**2 + Y**2)
    Z = np.sin(R)

    surf = p3.plot_surface(
        X+10, 
        Z-10, 
        Y+5)
    p3.setup_material_widgets(mesh=surf, tab=tab, index=index)
    return surf

def plot_all(kb=False, ts=False, wp=True, show_ui=True):
    k=None
    w=None
    s=None
    p3.clear()
    p3.figure()
    tab = widgets.Tab() if show_ui == True else None
    index = 0

    if kb:
        k=klein_bottle(tab=tab, index=index)
        index+=1
    if ts:
        s=test_surface(tab=tab, index=index)
        index+=1
    if wp:
        w=worldplane(tab=tab, index=index)
    p3.show()
    display(tab)
    return [k,w,s] 

def lights_dir_spot():
    dir1=p3.directional_light(light_color='green', 
        intensity=1.0,
        position=[30,30,30],
        target=[0,20,30])
    sp1=p3.spot_light(light_color='yellow',
        intensity=1,
        position=[20,30,20],
        target=[-20,-20,-20],
        angle=math.pi/9,
        penumbra=0.5)

# Ambient Light

In [2]:
plot_all(kb=True, ts=False, wp=True, show_ui=False)
tab = widgets.Tab()
light = p3.ambient_light(light_color="green")
p3.setup_light_widgets(light=light, tab=tab, index=0)
display(tab)

None

# Hemisphere Light

In [3]:
plot_all(kb=True, ts=False, wp=True, show_ui=False)
tab = widgets.Tab()
light = p3.hemisphere_light()
p3.setup_light_widgets(light=light, tab=tab, index=0)
display(tab)

None

# Directional Light

In [4]:
plot_all(kb=True, ts=False, wp=True, show_ui=False)
tab = widgets.Tab()
light_d = p3.directional_light(light_color="orange")
p3.setup_light_widgets(light=light_d, tab=tab, index=0)
display(tab)

None

# Spot Light

In [5]:
plot_all(kb=True, ts=False, wp=True, show_ui=False)
tab = widgets.Tab()
light_s = p3.spot_light(light_color="blue")
p3.setup_light_widgets(light=light_s, tab=tab, index=0)
display(tab)

None

# Point Light

In [6]:
plot_all(kb=True, ts=False, wp=True, show_ui=False)
tab = widgets.Tab()
light_p = p3.point_light(light_color="yellow")
p3.setup_light_widgets(light=light_p, tab=tab, index=0)
display(tab)

None

# Material properties

In [5]:
[bottle, plane, surf] = plot_all(kb=True, ts=True, wp=True, show_ui=False)
lights_dir_spot()
p3.show_lighting_widgets()

None

# Animated Light

In [11]:
[b1, p1, s1] = plot_all(kb=True, wp=True, ts=False)
point1=p3.point_light()

In [37]:
# Light and shadow Setup
point1.position_x=20
point1.position_y=10
point1.position_z=0
point1.cast_shadow = True
b1.cast_shadow = True
b1.receive_shadow = True
p1.receive_shadow = True
point1.shadow_map_type='PCF_SOFT'
point1.shadow_map_size=1024
# point1.shadow_radius=20 
point1.decay = 2

In [38]:
# Generating gif with animated light position
def set_pos(fig, i, fraction):
    point1.position_x = np.sin(2*np.pi*fraction) * 20
    point1.position_z = np.cos(2*np.pi*fraction) * 20
    
# a = p3.movie('rotatelight.gif', set_pos, fps=20, frames=40, endpoint=False, cmd_template_gif)

<img src="./rotatelight.gif" width="400" align="center">

In [ ]:
# WORK IN PROGRESS
# No need to change the lighting model manually
# whenever a light is added to the figure, the lighting model of the meshes will
# be updated to 'PHYSICAL'
#surf.lighting_model = 'PHYSICAL'
#plane.lighting_model = 'PHYSICAL'
#surf.color = 'blue'
#plane.color = 'blue'
#plane.emissive_intensity = 10
# Emissive (light) color of the material, essentially a solid color unaffected by other lighting. Default is black.

# roughness - How rough the material appears. 0.0 means a smooth mirror reflection, 1.0 means fully diffuse. 
# Default is 1.0.
# metalness - How much the material is like a metal. Non-metallic materials such as wood or stone use 0.0, 
# metallic use 1.0, with nothing (usually) in between. 
# Default is 0.0. A value between 0.0 and 1.0 could be used for a rusty metal look.
# PHYISICAL ONLY

# 1.0 - completely opaque; 0.0 - completely transparent

# specular_color - A specular highlight is the bright spot of light that appears on shiny objects when illuminated
# shininess - specular intensity
# PHONG ONLY
# Bui Tuong Phong, Illumination of Computer-Generated Images, Department of Computer Science, University of Utah, UTEC-CSs-73-129, July 1973.
# Bui Tuong Phong, "Illumination for Computer Generated Pictures," Comm. ACM, Vol 18(6):311-317, June 1975.

# Meshes can cast shadows and/or receive shadows

# Lambertian reflectance is the property that defines an ideal "matte" or diffusely reflecting surface.

#####################################################
# AMBIENT LIGHT
# light_color - default white
# intensity - default  1

#####################################################
# HEMISPHERE LIGHT
# light_color - default white - upper light
# light_color2 - default white - bottom light
# intensity - default  1
# position_x, position_y, position_z - default [0, 1, 0] 

#####################################################
# DIRECTIONAL LIGHT
# light_color - default white
# intensity - default  1
# position_x, position_y, position_z - default [0, 1, 0] 
# target_x, target_y, target_z - default [0, 0, 0]
# cast_shadow - default False
# shadow_map_size - default 512
# shadow_bias - default -0.0005
# shadow_radius - default 1
# shadow_camera_near - default 0.5
# shadow_camera_far - default 500
# shadow_camera_orthographic_size - default 100
# shadow_map_type - default 'PCF_SOFT'  - 'BASIC', 'PCF', 'PCF_SOFT'

# if some shadow settings do not update, due to a THREE.JS issue
# the settings should be updated when setting cast_shadow to False, Run cell, turn back to True and Run cell again
#directional1.cast_shadow=True
#directional1.shadow_map_size=512
#directional1.shadow_map_type='PCF_SOFT'
# Setting this to values greater than 1 will blur the edges of the shadow.
# High values will cause unwanted banding effects in the shadows - a greater mapSize will allow for a higher value to be used here before these effects become visible.
# If shadow_map_type is set to PCF_SOFT, shadow_radius has no effect and it is recommended to increase softness by decreasing mapSize instead.
#directional1.shadow_radius = 5 
#solves shadow acne issues
#directional1.shadow_bias=-0.005
#directional1.shadow_camera_orthographic_size=100
#####################################################
# POINT LIGHT
# light_color - default white
# intensity - default 1
# position_x, position_y, position_z - default [0, 1, 0]
# distance - default 0
# decay - default 1
# cast_shadow - default False
# shadow_map_size - default 512
# shadow_bias - default -0.0005
# shadow_radius - default 1
# shadow_camera_near - default 0.5
# shadow_camera_far - default 500
# shadow_map_type - default 'PCF_SOFT'  - 'BASIC', 'PCF', 'PCF_SOFT'
# point1.distance = 100 # Maximum range of the light. Default is 0 (no limit).
# # The amount the light dims along the distance of the light. 
# # Default is 1. For physically correct lighting, set this to 2.
# point1.decay = 0 
# # if some shadow settings do not update, due to a THREE.JS issue
# # the settings should be updated when setting cast_shadow to False, Run cell, turn back to True and Run cell again
# point1.cast_shadow=True
# point1.shadow_map_size=512
# point1.shadow_map_type='PCF'
# # Setting this to values greater than 1 will blur the edges of the shadow.
# # High values will cause unwanted banding effects in the shadows - a greater mapSize will allow for a higher value to be used here before these effects become visible.
# # If shadow_map_type is set to PCF_SOFT, shadow_radius has no effect and it is recommended to increase softness by decreasing mapSize instead.
# point1.shadow_radius=10 
# # solves shadow acne issues
# point1.shadow_bias=-0.005
# plane.color = 'red'
# plane.emissive_intensity = 10
# #surf.lighting_model = 'DEFAULT'
# #point1.intensity=1
# plane.color = 'green'
# plane.emissive_intensity = 100
#####################################################
# SPOT LIGHT
# light_color - default white
# intensity - default 1
# position_x, position_y, position_z - default [0, 1, 0]
# target_x, target_y, target_z - default [0, 0, 0]
# angle - default math.pi/3
# distance - default 0
# decay - default 1
# penumbra - default 0
# cast_shadow - default False
# shadow_map_size - default 512
# shadow_bias - default -0.0005
# shadow_radius - default 1
# shadow_camera_near - default 0.5
# shadow_camera_far - default 500
# shadow_camera_perspective_fov - default 50
# shadow_camera_perspective_aspect - default 1
# shadow_map_type - default 'PCF_SOFT' - 'BASIC', 'PCF', 'PCF_SOFT'

#plot_all(kb=True, wp=True, ts=False)
#spot1=p3.spot_light()

# Default mode — When distance is zero, light does not attenuate. 
# When distance is non-zero, light will attenuate linearly from maximum intensity at the 
# light's position down to zero at this distance from the light.
# Physically correct mode — When distance is zero, light will attenuate according to inverse-square law to infinite distance. 
# When distance is non-zero, light will attenuate according to inverse-square law until near the distance cutoff, 
# where it will then attenuate quickly and smoothly to 0. Inherently, cutoffs are not physically correct.
# Default is 0.0.
#spot1.distance=200

# The amount the light dims along the distance of the light.
# In physically correct mode, decay = 2 leads to physically realistic light falloff. The default is 1.
#spot1.decay=0

# Percent of the spotlight cone that is attenuated due to penumbra. 
# Takes values between zero and 1. The default is 0.0.
#spot1.penumbra=0.5

# If some shadow settings do not update, due to a THREE.JS issue
# the settings should be updated when setting cast_shadow to False, Run cell, turn back to True and Run cell again
#spot1.cast_shadow=True
#spot1.shadow_map_size=1024
#spot1.shadow_map_type='PCF_SOFT'

#spot1.shadow_camera_perspective_fov=50
# change the shadow camera aspect
#shadow_camera_perspective_aspect=1

# Setting this to values greater than 1 will blur the edges of the shadow.
# High values will cause unwanted banding effects in the shadows - a greater mapSize will allow for a higher value to be used here before these effects become visible.
# If shadow_map_type is set to PCF_SOFT, radius has no effect and it is recommended to increase softness by decreasing mapSize instead.
#spot1.shadow_radius=5 
# solves shadow acne issues
#spot1.shadow_bias=-0.0005